### 1) 라이브러리 설치

In [ ]:
# poetry add pypdf

Note: you may need to restart the kernel to use updated packages.


#### PyPDFLoader 간단한 예제

In [17]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_y


In [2]:
import os
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 파일 경로 설정
pdf_filepath = '../data/tutorial-korean.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")

try:
    # 1. PDF 파일 로드
    loader = PyPDFLoader(pdf_filepath)  # PDF 파일을 로드할 객체 생성
    docs = loader.load()  # 문서를 전체 로드

    # 총 문서 개수 출력
    print(f"총 {len(docs)}개의 문서가 로드 되었습니다.")

    #  첫 번째 문서의 메타데이터 출력
    print("첫 번째 문서 메타데이터:")
    print(json.dumps(docs[0].metadata, indent=2, ensure_ascii=False))

    # 특정 인덱스(10번째) 문서의 내용 확인 (존재할 경우)
    if len(docs) > 10:
        print("\n10번째 문서 내용:", type(docs[10]))
        print(docs[10])  # 10번째 문서 출력

    #  2. 텍스트 분할 (200자 단위, 중첩 없음)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    split_docs = loader.load_and_split(text_splitter=text_splitter)  # 분할된 문서 로드

    # 분할된 문서 개수 출력
    print(f"\n분할된 문서의 개수: {len(split_docs)} 타입: {type(split_docs)}")

    # 10번째 분할된 문서 내용 출력 (존재할 경우)
    if len(split_docs) > 10:
        print("\n10번째 분할된 문서:")
        print(split_docs[10])

    # 3. Lazy Load 방식으로 문서 로드
    print("\nLazy Load 방식으로 문서 로드:")
    for i, doc in enumerate(loader.lazy_load()):
        if i < 5:  # 너무 많은 출력 방지 (예제: 처음 5개만 출력)
            print(json.dumps(doc.metadata, indent=2, ensure_ascii=False))

except Exception as e:
    # 오류 발생 시 메시지 출력
    print(f"오류 발생: {e}")


총 39개의 문서가 로드 되었습니다.
첫 번째 문서 메타데이터:
{
  "producer": "Acrobat Distiller with ezUniHFT",
  "creator": "PScript5.dll Version 5.2",
  "creationdate": "2005-04-26T15:21:34+09:00",
  "moddate": "2005-04-26T15:21:34+09:00",
  "author": "Owner",
  "title": "<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>",
  "source": "../data/tutorial-korean.pdf",
  "total_pages": 39,
  "page": 0,
  "page_label": "1"
}

10번째 문서 내용: <class 'langchain_core.documents.base.Document'>
page_content='11그림 5 와 같이 getRoom과 setRoom 메소드들은 각각 staff 멤버의 방번호(room 
number)를 설정하고 반환하는 동작을 합니다. getRoom 메소드를 호출해 봅시다. 객체 메뉴의 getRoom 메소드를 선택하여 실행합니다. 그러면 대화상자에서 실행 결과를 볼 수 있을 것입니다( 그림 6 ). 그림 6 과 같이 결과의 내용이 "(unknown room)"이 됩니다. 왜냐하면, Staff 
객체에 대한 방번호를 지정하지 않았기 때문입니다.
                     
그림 6 : 메소드 호출 결과
슈퍼 클래스에서 상속된 메소드들은 서브 메뉴(inherited from Person)에서 선택하여 사용
할 수 있습니다. 객체 팝업메뉴의 상단에는 두개의 서브메뉴
3)가 있을 것입니다. 하나는 
Object 클래스로부터 상속 받은 메소드들이고 다른 하나는 Person 클래스로부터 상속 받은 메소드들입니다( 그림 5 ). 따라서, 서브 메뉴를 선택하면 getName과 같은 Per

#### OllamaEmbeddings
* bge-m3:latest 모델

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaEmbeddings

print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('../data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기 (한국어 최적화)
    chunk_overlap=200,      # 중복 부분 (맥락 보존)
    separators=["\n\n", "\n", ".", " ", ""] # 자연스러운 분할을 위한 구분자
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("==> 3. 벡터화 → 임베딩으로 변환")
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=1536
# )

#좀 느림
embeddings = OllamaEmbeddings(model="bge-m3:latest") 

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

# --------------------------
vectorstore.save_local("faiss_db")
# --------------------------
print("="*20)
print("===> 5. 검색 → 질문과 유사한 문서 찾기")

#VectorStoreRetriever 부분
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # 상위 6개 관련 문서 검색
)
print(" Retriever 설정 완료")

print("===> 6. 생성 → LLM으로 답변 생성")
# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0.1,
#     max_tokens=1500
# )
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")

# ===================================
# 7. QA 체인 생성 -> Retriver+LLm을 체인으로 연결
# ===================================
print("\n ===> 7.  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("  RAG 파이프라인 구축 완료!\n",'-'*50)




==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  66개 청크 생성 완료
  평균 청크 길이: 688자
==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (66개 벡터)
===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료
===> 6. 생성 → LLM으로 답변 생성
 프롬프트 설정 완료

 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!
 --------------------------------------------------
{'query': '애플릿을 만들고 실행하는 방법을 설명해주세요', 'result': 'BlueJ에서 애플릿을 만들고 실행하는 방법은 다음과 같습니다.\n\n1. 애플릿 실행하기  \n   1) 예제 디렉토리에서 appletdemo 프로젝트를 엽니다.  \n   2) CaseConverter 클래스(아이콘에 <<applet>> 태그가 표시됨)를 컴파일합니다.  \n   3) 클래스 팝업 메뉴에서 “Run Applet” 명령을 선택합니다.  \n   4) “Run Applet” 대화상자에서 브라우저 실행 여부 등을 선택한 뒤 OK 버튼을 클릭하면 애플릿뷰어가 애플릿을 화면에 보여줍니다.\n\n2. 애플릿 만들기  \n   1) “New Class” 버튼을 클릭합니다.  \n   2) New Class 대화상자에서 클래스 형식으로 “Applet”을 선택합니다.  \n   3) 컴파일한 뒤 애플릿을 실행합니다.  \n   4) 에디터를 열어 기본 골격 코드를 원하는 대로 수정합니다.\n\n3. 애플릿 테스트하기  \n   - 오브젝트 벤치에서 애플릿 객체를 생성할 수 있으며, 일부 메소드를 테스트할 수 있습니다.  \n   - 단, 애플릿뷰어나 브라우저에서 실행할 경우 BlueJ의 중단점은 동작하지 않습니다.  \n   - 중단점과 한 단계씩 실행

In [16]:
from pprint import pprint
pprint(result)

{'query': '애플릿을 만들고 실행하는 방법을 설명해주세요',
 'result': 'BlueJ에서 애플릿을 만들고 실행하는 방법은 다음과 같습니다.\n'
           '\n'
           '1. 애플릿 실행하기  \n'
           '   1) 예제 디렉토리에서 appletdemo 프로젝트를 엽니다.  \n'
           '   2) CaseConverter 클래스(아이콘에 <<applet>> 태그가 표시됨)를 컴파일합니다.  \n'
           '   3) 클래스 팝업 메뉴에서 “Run Applet” 명령을 선택합니다.  \n'
           '   4) “Run Applet” 대화상자에서 브라우저에서 실행할지 애플릿 뷰어에서 실행할지 선택한 뒤, 기본 설정으로 '
           'OK 버튼을 클릭합니다.  \n'
           '   5) 잠시 후 애플릿 뷰어가 CaseConverter 애플릿을 화면에 표시합니다.\n'
           '\n'
           '2. 애플릿 만들기  \n'
           '   1) “New Class” 버튼을 클릭합니다.  \n'
           '   2) New Class 대화상자에서 클래스 형식으로 “Applet”을 선택합니다.  \n'
           '   3) 컴파일한 후 애플릿을 실행합니다.  \n'
           '   4) 생성된 기본 골격에서 에디터를 열어 원하는 코드를 삽입·편집합니다.  \n'
           '   5) 주로 paint 메소드 안에 샘플 코드가 있으며, 필요에 따라 애플릿 메소드를 수정합니다.\n'
           '\n'
           '3. 애플릿 테스트하기  \n'
           '   - 오브젝트 벤치에서 애플릿 객체를 생성할 수 있으며, 일부 메소드를 테스트할 수 있습니다.  \n'
           '   - 단, 오브젝트 벤치에서는 애플릿을 완전히 실행할 수 없습니다. 

In [9]:
# ===================================
# 8. 테스트 질문들
# ===================================
test_questions = [
    "BlueJ에서 객체를 생성하는 방법은 무엇인가요?",
    "컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?", 
    "디버깅을 위해 중단점을 설정하는 방법을 알려주세요",
    "코드패드는 무엇이고 어떻게 사용하나요?",
    "애플릿을 만들고 실행하는 방법을 설명해주세요"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 9. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


 RAG 시스템 테스트

【테스트 1/5】
 질문: BlueJ에서 객체를 생성하는 방법은 무엇인가요?
 답변 생성 중...

 답변:
--------------------------------------------------
BlueJ에서 객체를 생성하는 방법은 다음과 같습니다.

1. 클래스 아이콘에서 마우스 오른쪽 버튼을 클릭합니다.  
2. 팝업 메뉴에서 원하는 생성자(constructor)를 선택합니다.  
3. 나타난 대화상자에 객체 이름을 입력합니다. (기본 이름이 제공되므로 그대로 사용해도 무방합니다.)  
4. OK 버튼을 클릭합니다.  
5. 생성된 객체가 오브젝트 벤치(object bench)에 표시됩니다.

 참조 문서:
   1. 페이지 9: 10이 대화상자는 생성할 객체의 이름을 입력받습니다. 또한 동시에 기본적인 이름(staff_1)이  제공됩니다. 이 이름(staff1)은 지금 ...
   2. 페이지 7: 83.2. 프로젝트 열기 프로젝트란, 관련 있는 여러개의 파일들을 묶어서 관리하기 위한 하나의 작업 단위를 말합니다.  BlueJ  프로젝트는 ...
   3. 페이지 35: 3610.6. 라이브러리 클래스로부터 객체생성하기 BlueJ 는 JDK에서 제공하는 라이브러리의 클래스들로부터 객체를 생성하기 위한 기능을  제...

----------------------------------------

【테스트 2/5】
 질문: 컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
컴파일 오류 확인 방법은 다음과 같습니다.

1. 에디터를 열고 소스 코드에 오류가 있는 상태에서 **Compile** 버튼을 누릅니다.  
2. 오류가 발생하면  
   - 오류가 난 줄이 반전되어 강조 표시됩니다.  
   - 에디터 하단의 정보창(information area)에 에러 메시지가 출력됩니다.  
3. 정보창 오른쪽